In [1]:
import numpy as np
import g4f
from time import sleep
from fuzzywuzzy import fuzz
from glob import glob
from datasets import Dataset
import numpy as np
import pandas as pd
from pprint import pprint
import os
import json

In [2]:
g4f.Provider.GeekGpt

g4f.Provider.GeekGpt.GeekGpt

In [3]:
def sample_pool(n, pool):
    if n >= len(pool):
        return pool
    return list(np.random.choice(pool, size=n, replace=False))


def sample_seed(n, seed):
    return list(np.random.choice(seed, size=n, replace=False))


TEMPLATE1 = """Tôi đang xây dựng module tìm kiếm trong một hệ thống TV trực tuyến chiếu các kênh truyền hình. Tưởng tượng bạn là một người dùng ra lệnh cho hệ thống, hãy sinh ra các template người dùng thực tế thường nhập vào để tìm kiếm kênh mà họ mong muốn.

Ý định của người dùng (user_intent) có thể là:
1) Mở trực tiếp kênh (mo_truc_tiep)
2) Xem lịch phát sóng (lich_phat_song)

Sau đây là một số ví dụ, hãy dựa vào các ví dụ này để xây dựng nhiều các template khác:
{templates}

"""
TEMPLATE2 = """Lưu ý: 
1) Các template cần đa dạng, khác nhau về nội dung, mô tả đúng các tình huống thực tế của người dùng.
2) Chỉ trả về các template dưới dạng json: [{'template': 'your_template', 'intent':'user_intent'}, ...] không giải thích gì thêm."""


def get_response(examples):
    while True:
        try:
            templates = "\n".join(examples)
            prompt = TEMPLATE1.format(templates=templates) + TEMPLATE2
            response = g4f.ChatCompletion.create(
                provider=g4f.Provider.GeekGpt,
                model=g4f.models.gpt_4,
                messages=[{"role": "user", "content": prompt}],
            )
            return response
        except Exception as e:
            print(e)
            print("Request failed")
            sleep(5)


def extract_json(response):
    start = response.find("[")
    end = response.find("]") + 1
    if start == -1 or end == -1:
        return None
    try:
        return eval(response[start:end])
    except Exception as e:
        return None


def remove_duplicates(l, threshold=80):
    final = []
    for i in l:
        can_add = True
        for j in final:
            score = fuzz.ratio(i["template"], j["template"])
            if score >= threshold:
                can_add = False
                break
        if can_add:
            final.append(i)
    print(
        f"Removed {len(l) - len(final)} similar questions from total {len(l)} questions."
    )
    return final


def generate_questions(
    n_rounds=20,
    seed_path="./channel_seed.txt",
    pool_path="./channel_pool.txt",
    threshold=90,
    save_steps=5,
    n_seed=4,
    n_pool=6,
):
    for i in range(n_rounds):
        question_pool = []
        with open(pool_path, "r") as f:
            for line in f:
                question_pool.append(eval(line.strip())["template"])

        with open(seed_path, "r") as f:
            seed_tasks = f.read().splitlines()

        print(f"Round {i+1}")
        seed = sample_seed(n_seed, seed_tasks)
        pool = sample_pool(n_pool, question_pool)
        response = get_response(seed + pool)
        js = extract_json(response)
        if js is None:
            print("No json found")
            continue
        with open(pool_path, "a") as f:
            for q in js:
                f.write(str(q) + "\n")

        if (i + 1) % save_steps == 0:
            with open(pool_path, "r") as f:
                lines = f.read().splitlines()
                lines = [eval(line.strip()) for line in lines]
            lines = remove_duplicates(lines, threshold=threshold)

            with open(pool_path, "w") as f:
                for line in lines:
                    f.write(str(line) + "\n")

In [4]:
generate_questions(50)

Round 1
New pypi version: 0.1.9.6 (current: 0.1.9.3) | pip install -U g4f
Round 2
429 Client Error: Too Many Requests for url: https://ai.fakeopen.com/v1/chat/completions
Request failed
Round 3
429 Client Error: Too Many Requests for url: https://ai.fakeopen.com/v1/chat/completions
Request failed
429 Client Error: Too Many Requests for url: https://ai.fakeopen.com/v1/chat/completions
Request failed
429 Client Error: Too Many Requests for url: https://ai.fakeopen.com/v1/chat/completions
Request failed
Round 4
429 Client Error: Too Many Requests for url: https://ai.fakeopen.com/v1/chat/completions
Request failed
429 Client Error: Too Many Requests for url: https://ai.fakeopen.com/v1/chat/completions
Request failed
No json found
Round 5
429 Client Error: Too Many Requests for url: https://ai.fakeopen.com/v1/chat/completions
Request failed
429 Client Error: Too Many Requests for url: https://ai.fakeopen.com/v1/chat/completions
Request failed


KeyboardInterrupt: 